In [7]:
import numpy as np
import random
import time

In [8]:
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 10
NUM_ITEMS = 100
NUM_DIMENSIONS = 10

VALUES = rng.integers(0, 1000, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 1000, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(
    1000 * 2, 1000 * NUM_ITEMS // NUM_KNAPSACKS, size=(NUM_KNAPSACKS, NUM_DIMENSIONS)
)

print("VALUES shape:", VALUES.shape)
print("WEIGHTS shape:", WEIGHTS.shape)
print("CONSTRAINTS shape:", CONSTRAINTS.shape)

VALUES shape: (100,)
WEIGHTS shape: (100, 10)
CONSTRAINTS shape: (10, 10)


In [ ]:
def evaluate(solution, values, weights, constraints):

    num_knapsacks, num_dims = constraints.shape
    loads = np.zeros((num_knapsacks, num_dims), dtype=int)
    total_value = 0
    for i, k in enumerate(solution):
        if k > 0:
            loads[k - 1] += weights[i]
            total_value += values[i]
    feasible = np.all(loads <= constraints)
    violation = np.maximum(0, loads - constraints).sum()
    return total_value, loads, feasible, violation

In [10]:
def greedy_initial(values, weights, constraints):
    num_items = len(values)
    num_knapsacks, num_dims = constraints.shape
    order = sorted(range(num_items),
                   key=lambda i: values[i] / (1 + np.sum(weights[i])),
                   reverse=True)
    loads = np.zeros((num_knapsacks, num_dims), dtype=int)
    sol = [0] * num_items

    for i in order:
        for k in range(num_knapsacks):
            if np.all(loads[k] + weights[i] <= constraints[k]):
                sol[i] = k + 1
                loads[k] += weights[i]
                break
    return sol

In [ ]:
def tabu_search(values, weights, constraints,
                max_iters=3000,
                tabu_tenure=25,
                neighbor_size=300,
                penalty_lambda=100.0):

    # Initialization
    current = greedy_initial(values, weights, constraints)
    curr_val, _, curr_feas, curr_viol = evaluate(current, values, weights, constraints)
    curr_score = curr_val - penalty_lambda * curr_viol

    best_feas_sol = current.copy() if curr_feas else None
    best_feas_val = curr_val if curr_feas else -1
    best_sol = current.copy()
    best_score = curr_score

    tabu_list = {}  # key: (item, knapsack), value: iteration until tabu expires
    num_items = len(values)
    num_knapsacks = constraints.shape[0]
    iteration = 0
    start_time = time.time()

    # Main loop
    while iteration < max_iters:
        iteration += 1
        best_move = None
        best_move_score = -1e12
        best_move_solution = None

        # Generate candidate neighbors
        for _ in range(neighbor_size):
            i = random.randrange(num_items)
            new_k = random.randrange(0, num_knapsacks + 1)
            if new_k == current[i]:
                continue
            neighbor = current.copy()
            neighbor[i] = new_k
            val, _, feas, viol = evaluate(neighbor, values, weights, constraints)
            score = val - penalty_lambda * viol
            move = (i, new_k)

            # Tabu check (with aspiration criterion)
            is_tabu = move in tabu_list and tabu_list[move] > iteration
            if is_tabu and score <= best_feas_val:
                continue  # tabu and not aspirational

            if score > best_move_score:
                best_move_score = score
                best_move_solution = neighbor
                best_move = move

        # If no neighbor found, break
        if best_move_solution is None:
            break

        # Apply best move
        current = best_move_solution
        curr_val, _, curr_feas, curr_viol = evaluate(current, values, weights, constraints)
        curr_score = curr_val - penalty_lambda * curr_viol

        # Update Tabu list
        tabu_list[best_move] = iteration + tabu_tenure

        # Keep best found
        if curr_score > best_score:
            best_score = curr_score
            best_sol = current.copy()

        if curr_feas and curr_val > best_feas_val:
            best_feas_val = curr_val
            best_feas_sol = current.copy()

        # periodically decay tabu list (clean expired entries)
        if iteration % 50 == 0:
            tabu_list = {k: v for k, v in tabu_list.items() if v > iteration}

        # Print progress occasionally
        if iteration % 500 == 0:
            print(f"Iter {iteration}: Best feasible={best_feas_val}, Current={curr_val}, Viol={curr_viol}")

    elapsed = time.time() - start_time
    return {
        "best_feasible_value": best_feas_val,
        "best_feasible_sol": best_feas_sol,
        "best_overall_value": best_score,
        "best_overall_sol": best_sol,
        "time": elapsed
    }

In [12]:
result = tabu_search(VALUES, WEIGHTS, CONSTRAINTS)

print("\n--- TABU SEARCH RESULT ---")
print("Best feasible value:", result["best_feasible_value"])
print("Best feasible solution (first 30 items):", result["best_feasible_sol"][:30], "...")
val, loads, feas, viol = evaluate(result["best_feasible_sol"], VALUES, WEIGHTS, CONSTRAINTS)
print("Feasible:", feas, "Violation:", viol)
print("Loads per knapsack (first 3 shown):\n", loads[:3])
print("Runtime (seconds):", result["time"])

Iter 500: Best feasible=49713, Current=49242, Viol=0
Iter 1000: Best feasible=50540, Current=49935, Viol=2
Iter 1500: Best feasible=50963, Current=49737, Viol=0
Iter 2000: Best feasible=51177, Current=50244, Viol=0
Iter 2500: Best feasible=51177, Current=50656, Viol=0
Iter 3000: Best feasible=51177, Current=50269, Viol=0

--- TABU SEARCH RESULT ---
Best feasible value: 51177
Best feasible solution (first 30 items): [0, 9, 9, 10, 7, 5, 0, 3, 6, 0, 4, 6, 3, 10, 4, 3, 5, 10, 6, 9, 4, 5, 4, 2, 5, 3, 6, 4, 7, 10] ...
Feasible: True Violation: 0
Loads per knapsack (first 3 shown):
 [[4128 4972 2862 3198 4443 3342 3532 3691 3124 1980]
 [5652 3722 3061 4192 3297 5320 5990 3936 4098 4939]
 [1978 2535 3959 3762 5430 4699 4900 3916 4266 5121]]
Runtime (seconds): 363.0171239376068
